# MIAO Automatic Modeling: Depression Severity Detection

This notebook demonstrates automatic generation of MIAO-compliant RDF annotations from:
1. A text dataset with ordinal depression severity labels (0=Minimal, 1=Mild, 2=Moderate, 3=Severe)
2. Machine learning experiment results from multiple models (BERT, Feature Framework)

The notebook creates a complete RDF knowledge graph following the MIAO ontology structure for depression severity research.

## 1. Setup and Dependencies

In [1]:
# Install required packages (uncomment if needed)
# !pip install rdflib pandas numpy scikit-learn

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from rdflib import Graph, Namespace, Literal, URIRef, RDF, RDFS, XSD
from rdflib.namespace import DCTERMS, PROV
import hashlib
import json

## 2. Define Namespaces

In [3]:
# Define MIAO and related namespaces
MIAO = Namespace("https://w3id.org/miao#")
MLS = Namespace("http://www.w3.org/ns/mls#")
EX = Namespace("https://w3id.org/miao/depression-experiment#")

# Create RDF graph
g = Graph()
g.bind("miao", MIAO)
g.bind("mls", MLS)
g.bind("ex", EX)
g.bind("dcterms", DCTERMS)
g.bind("prov", PROV)
g.bind("rdfs", RDFS)
g.bind("xsd", XSD)

## 3. Load Input Data

### 3.1 Load Depression Severity Dataset

Expected format:
- CSV file with columns: `text`, `severity_label`
- `severity_label`: 0 (Minimal), 1 (Mild), 2 (Moderate), 3 (Severe)

In [4]:
# Load dataset (replace with your actual file path)
# For demonstration, we'll create a sample dataset

# Option 1: Load from CSV
# df_dataset = pd.read_csv('depression_severity_dataset.csv')

# Option 2: Create sample data for demonstration
df_dataset = pd.DataFrame({
    'text': [
        "Feeling okay today, nothing special but managing fine",
        "Sometimes I feel a bit down but it passes quickly",
        "Having trouble getting out of bed, everything feels pointless",
        "I can't remember the last time I felt happy, constant sadness",
        "Life is good, enjoying my hobbies and social activities",
        "Occasional low mood but generally functioning well",
        "Struggling with daily tasks, feeling hopeless about the future",
        "Severe symptoms affecting my work, relationships, and self-care",
        "Minimal symptoms, coping well with stress",
        "Moderate depression interfering with some aspects of my life"
    ],
    'severity_label': [0, 1, 2, 3, 0, 1, 2, 3, 0, 2]  # 0=Minimal, 1=Mild, 2=Moderate, 3=Severe
})

# Map labels to severity names
severity_map = {0: 'Minimal', 1: 'Mild', 2: 'Moderate', 3: 'Severe'}
df_dataset['severity_name'] = df_dataset['severity_label'].map(severity_map)

print(f"Loaded {len(df_dataset)} samples")
print(f"\nSeverity distribution:")
print(df_dataset['severity_name'].value_counts().sort_index())
df_dataset.head()

Loaded 10 samples

Severity distribution:
severity_name
Mild        2
Minimal     3
Moderate    3
Severe      2
Name: count, dtype: int64


,text,severity_label,severity_name
0,"Feeling okay today, nothing special but managi...",0,Minimal
1,Sometimes I feel a bit down but it passes quickly,1,Mild
2,"Having trouble getting out of bed, everything ...",2,Moderate
3,"I can't remember the last time I felt happy, c...",3,Severe
4,"Life is good, enjoying my hobbies and social a...",0,Minimal


### 3.2 Load ML Experiment Results

Expected format:
- DataFrame with columns: `sample_id`, `model_name`, `predicted_label`, `confidence`, `true_label`
- Support for multiple models (BERT, Feature Framework, etc.)

In [5]:
# Option 1: Load from CSV
# df_results = pd.read_csv('experiment_results.csv')

# Option 2: Create sample results for demonstration (multiple models)
np.random.seed(42)

# BERT model results
bert_results = pd.DataFrame({
    'sample_id': range(10),
    'model_name': 'BERT_Depression_Classifier',
    'predicted_label': [0, 1, 2, 3, 0, 1, 2, 3, 0, 2],
    'confidence': [0.89, 0.76, 0.82, 0.91, 0.93, 0.71, 0.78, 0.88, 0.90, 0.80],
    'true_label': df_dataset['severity_label'].values
})

# Feature Framework results
framework_results = pd.DataFrame({
    'sample_id': range(10),
    'model_name': 'Feature_Framework_Classifier',
    'predicted_label': [0, 1, 2, 3, 0, 1, 2, 2, 0, 2],  # One misclassification
    'confidence': [0.85, 0.79, 0.84, 0.87, 0.91, 0.74, 0.81, 0.75, 0.88, 0.83],
    'true_label': df_dataset['severity_label'].values
})

df_results = pd.concat([bert_results, framework_results], ignore_index=True)

# Model performance metrics (aggregate)
model_metrics = {
    'BERT_Depression_Classifier': {
        'implementation': 'PyTorch',
        'version': '1.0',
        'architecture': 'BERT-base fine-tuned',
        'accuracy': 0.90,
        'macro_precision': 0.88,
        'macro_recall': 0.89,
        'macro_f1': 0.88,
        'weighted_f1': 0.90,
        'training_date': '2025-11-20',
        'hyperparameters': {
            'learning_rate': 2e-5,
            'batch_size': 32,
            'epochs': 10,
            'max_length': 512
        }
    },
    'Feature_Framework_Classifier': {
        'implementation': 'scikit-learn',
        'version': '1.0',
        'architecture': 'SVM with LIWC features and GloVe embeddings',
        'accuracy': 0.85,
        'macro_precision': 0.83,
        'macro_recall': 0.84,
        'macro_f1': 0.83,
        'weighted_f1': 0.85,
        'training_date': '2025-11-20',
        'hyperparameters': {
            'kernel': 'rbf',
            'C': 1.0,
            'gamma': 'scale',
            'embedding_dim': 300
        }
    }
}

print(f"Loaded {len(df_results)} predictions from {df_results['model_name'].nunique()} models")
print(f"\nAccuracy by model:")
for model in df_results['model_name'].unique():
    model_data = df_results[df_results['model_name'] == model]
    acc = (model_data['predicted_label'] == model_data['true_label']).mean()
    print(f"  {model}: {acc:.2%}")

df_results.head(10)

Loaded 20 predictions from 2 models

Accuracy by model:
  BERT_Depression_Classifier: 100.00%
  Feature_Framework_Classifier: 90.00%


,sample_id,model_name,predicted_label,confidence,true_label
0,0,BERT_Depression_Classifier,0,0.89,0
1,1,BERT_Depression_Classifier,1,0.76,1
2,2,BERT_Depression_Classifier,2,0.82,2
3,3,BERT_Depression_Classifier,3,0.91,3
4,4,BERT_Depression_Classifier,0,0.93,0
5,5,BERT_Depression_Classifier,1,0.71,1
6,6,BERT_Depression_Classifier,2,0.78,2
7,7,BERT_Depression_Classifier,3,0.88,3
8,8,BERT_Depression_Classifier,0,0.90,0
9,9,BERT_Depression_Classifier,2,0.80,2


## 4. Create MIAO Depression Severity Schema

In [6]:
def create_depression_severity_schema(graph):
    """
    Create ordinal depression severity classification schema in MIAO format.
    """
    # Define schema
    schema_uri = EX.DepressionSeveritySchema_Research
    graph.add((schema_uri, RDF.type, MIAO.MentalIllnessesSchema))
    graph.add((schema_uri, DCTERMS.title, 
               Literal("Research schema for depression severity in text", lang="en")))
    graph.add((schema_uri, DCTERMS.description, 
               Literal("Ordinal taxonomy of depression severity (Minimal, Mild, Moderate, Severe) used in social media corpora for computational research on depression detection", lang="en")))
    graph.add((schema_uri, DCTERMS.created, 
               Literal(datetime.now().strftime("%Y-%m-%d"), datatype=XSD.date)))
    
    # Define categories with PHQ-9 score ranges
    categories = [
        (0, "Minimal", "Minimal or no depressive symptoms. PHQ-9 score range: 0-4."),
        (1, "Mild", "Mild depressive symptoms. PHQ-9 score range: 5-9."),
        (2, "Moderate", "Moderate depressive symptoms. PHQ-9 score range: 10-14."),
        (3, "Severe", "Severe depressive symptoms. PHQ-9 score range: 15-27.")
    ]
    
    category_map = {}
    
    for level, name, description in categories:
        category_uri = EX[name]
        graph.add((category_uri, RDF.type, MIAO.MentalIllnessCategory))
        graph.add((category_uri, DCTERMS.title, Literal(f"{name} depression", lang="en")))
        graph.add((category_uri, DCTERMS.description, Literal(description, lang="en")))
        graph.add((category_uri, DCTERMS.identifier, Literal(str(level), datatype=XSD.integer)))
        graph.add((category_uri, MIAO.isMentalIllnessCategoryOf, schema_uri))
        graph.add((schema_uri, MIAO.hasMentalIllnessCategory, category_uri))
        graph.add((category_uri, RDFS.label, Literal(f"{name} Depression", lang="en")))
        
        category_map[level] = category_uri
    
    return schema_uri, category_map

schema_uri, category_map = create_depression_severity_schema(g)
print(f"Created schema: {schema_uri}")
print(f"\nCategories:")
for level, uri in category_map.items():
    print(f"  {level}: {uri}")

Created schema: https://w3id.org/miao/depression-experiment#DepressionSeveritySchema_Research

Categories:
  0: https://w3id.org/miao/depression-experiment#Minimal
  1: https://w3id.org/miao/depression-experiment#Mild
  2: https://w3id.org/miao/depression-experiment#Moderate
  3: https://w3id.org/miao/depression-experiment#Severe


## 5. Model Dataset as MIAO Dataset

In [7]:
def create_dataset_metadata(graph, df, dataset_name="DepressionSeverityTextDataset"):
    """
    Create dataset metadata in MLS format.
    """
    dataset_uri = EX[dataset_name]
    graph.add((dataset_uri, RDF.type, MLS.Dataset))
    graph.add((dataset_uri, RDFS.label, 
               Literal(f"{dataset_name} - Social media text for depression severity detection", lang="en")))
    graph.add((dataset_uri, DCTERMS.description, 
               Literal(f"Dataset containing {len(df)} social media text samples with ordinal depression severity annotations (Minimal, Mild, Moderate, Severe)", lang="en")))
    graph.add((dataset_uri, DCTERMS.extent, 
               Literal(f"{len(df)} samples", lang="en")))
    graph.add((dataset_uri, DCTERMS.format, Literal("text/plain")))
    graph.add((dataset_uri, DCTERMS.created, 
               Literal(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"), datatype=XSD.dateTime)))
    graph.add((dataset_uri, DCTERMS.source, Literal("Reddit (r/depression, r/mentalhealth)", lang="en")))
    
    # Add severity distribution statistics
    for level in range(4):
        count = (df['severity_label'] == level).sum()
        severity_name = severity_map[level]
        graph.add((dataset_uri, EX[f"{severity_name.lower()}Count"], 
                   Literal(int(count), datatype=XSD.integer)))
    
    return dataset_uri

dataset_uri = create_dataset_metadata(g, df_dataset)
print(f"Created dataset: {dataset_uri}")

Created dataset: https://w3id.org/miao/depression-experiment#DepressionSeverityTextDataset


## 6. Model ML Implementations and Models

In [8]:
def create_ml_implementation(graph, model_name, metrics):
    """
    Create ML implementation and software metadata for a specific model.
    """
    impl_type = metrics['implementation']
    
    # Software (PyTorch or scikit-learn)
    software_uri = EX[impl_type.replace('-', '_')]
    graph.add((software_uri, RDF.type, MLS.Software))
    graph.add((software_uri, RDFS.label, Literal(impl_type, lang="en")))
    
    if impl_type == "PyTorch":
        graph.add((software_uri, DCTERMS.description, 
                   Literal("Open-source machine learning framework", lang="en")))
        graph.add((software_uri, DCTERMS.hasVersion, Literal("2.1.0")))
    else:
        graph.add((software_uri, DCTERMS.description, 
                   Literal("Machine learning library for Python", lang="en")))
        graph.add((software_uri, DCTERMS.hasVersion, Literal("1.3.0")))
    
    # Implementation
    impl_name = model_name.replace(' ', '_')
    impl_uri = EX[f"{impl_name}_Implementation"]
    graph.add((impl_uri, RDF.type, MLS.Implementation))
    graph.add((impl_uri, RDFS.label, 
               Literal(f"{model_name} Implementation", lang="en")))
    graph.add((impl_uri, DCTERMS.description, 
               Literal(metrics['architecture'], lang="en")))
    graph.add((software_uri, MLS.hasPart, impl_uri))
    
    # Hyperparameters
    for param_name, param_value in metrics['hyperparameters'].items():
        param_uri = EX[f"{impl_name}_{param_name}"]
        graph.add((param_uri, RDF.type, MLS.HyperParameter))
        graph.add((param_uri, RDFS.label, 
                   Literal(param_name.replace('_', ' ').title(), lang="en")))
        
        # Determine datatype
        if isinstance(param_value, float):
            graph.add((param_uri, MLS.hasValue, Literal(param_value, datatype=XSD.float)))
        elif isinstance(param_value, int):
            graph.add((param_uri, MLS.hasValue, Literal(param_value, datatype=XSD.integer)))
        else:
            graph.add((param_uri, MLS.hasValue, Literal(str(param_value))))
        
        graph.add((impl_uri, MLS.hasHyperParameter, param_uri))
    
    return impl_uri, software_uri

def create_trained_model(graph, model_name, metrics, impl_uri):
    """
    Create trained model instance.
    """
    model_name_clean = model_name.replace(' ', '_')
    model_uri = EX[f"{model_name_clean}_v{metrics['version']}"]
    graph.add((model_uri, RDF.type, MLS.Model))
    graph.add((model_uri, RDFS.label, 
               Literal(f"{model_name} v{metrics['version']}", lang="en")))
    graph.add((model_uri, DCTERMS.created, 
               Literal(metrics['training_date'], datatype=XSD.date)))
    graph.add((model_uri, DCTERMS.description, 
               Literal(f"Trained model for 4-class depression severity classification: {metrics['architecture']}", lang="en")))
    
    return model_uri

# Create implementations and models for all models
implementations = {}
models = {}

for model_name, metrics in model_metrics.items():
    impl_uri, software_uri = create_ml_implementation(g, model_name, metrics)
    model_uri = create_trained_model(g, model_name, metrics, impl_uri)
    implementations[model_name] = impl_uri
    models[model_name] = model_uri
    print(f"Created implementation and model for: {model_name}")

print(f"\nTotal implementations created: {len(implementations)}")
print(f"Total models created: {len(models)}")

Created implementation and model for: BERT_Depression_Classifier
Created implementation and model for: Feature_Framework_Classifier

Total implementations created: 2
Total models created: 2


## 7. Model Detection Runs and Evaluations

In [9]:
def create_detection_run(graph, model_name, schema_uri, dataset_uri, impl_uri, model_uri, metrics):
    """
    Create automatic detection run (experiment execution) for a specific model.
    """
    run_id = hashlib.md5(f"{model_name}_{datetime.now()}".encode()).hexdigest()[:8]
    run_uri = EX[f"Run_{model_name.replace(' ', '_')}_{run_id}"]
    
    graph.add((run_uri, RDF.type, MIAO.AutomaticMentalIllnessesDetection))
    graph.add((run_uri, RDF.type, MLS.Run))
    graph.add((run_uri, RDFS.label, 
               Literal(f"Depression severity detection run: {model_name}", lang="en")))
    graph.add((run_uri, DCTERMS.description, 
               Literal(f"Automatic depression severity detection using {model_name}", lang="en")))
    graph.add((run_uri, DCTERMS.created, 
               Literal(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"), datatype=XSD.dateTime)))
    
    # Connect to components
    graph.add((run_uri, MIAO.hasInputData, dataset_uri))
    graph.add((run_uri, MIAO.usedMentalIllnessesSchema, schema_uri))
    graph.add((run_uri, MLS.executes, impl_uri))
    
    # Create evaluation measures and metrics
    evaluation_metrics = {
        'accuracy': ('Accuracy', metrics['accuracy']),
        'macro_precision': ('Macro Precision', metrics['macro_precision']),
        'macro_recall': ('Macro Recall', metrics['macro_recall']),
        'macro_f1': ('Macro F1 Score', metrics['macro_f1']),
        'weighted_f1': ('Weighted F1 Score', metrics['weighted_f1'])
    }
    
    for metric_key, (metric_label, metric_value) in evaluation_metrics.items():
        # Create or reference evaluation measure
        measure_uri = EX[metric_key]
        if (measure_uri, RDF.type, MLS.EvaluationMeasure) not in graph:
            graph.add((measure_uri, RDF.type, MLS.EvaluationMeasure))
            graph.add((measure_uri, RDFS.label, Literal(metric_label, lang="en")))
        
        # Create evaluation instance
        eval_uri = EX[f"{run_id}_{metric_key}_evaluation"]
        graph.add((eval_uri, RDF.type, MLS.ModelEvaluation))
        graph.add((eval_uri, MLS.specifiedBy, measure_uri))
        graph.add((eval_uri, MLS.hasValue, Literal(metric_value, datatype=XSD.float)))
        graph.add((run_uri, MLS.hasOutput, eval_uri))
    
    graph.add((run_uri, MLS.hasOutput, model_uri))
    
    return run_uri, run_id

# Create detection runs for all models
runs = {}
run_ids = {}

for model_name, metrics in model_metrics.items():
    run_uri, run_id = create_detection_run(
        g, model_name, schema_uri, dataset_uri, 
        implementations[model_name], models[model_name], metrics
    )
    runs[model_name] = run_uri
    run_ids[model_name] = run_id
    print(f"Created detection run for: {model_name} (ID: {run_id})")

Created detection run for: BERT_Depression_Classifier (ID: 7c8492f1)
Created detection run for: Feature_Framework_Classifier (ID: f4a4e505)


## 8. Model Individual Predictions

In [10]:
def create_predictions(graph, df_results, df_dataset, model_name, run_uri, run_id, category_map):
    """
    Create individual mental illness predictions for each sample from a specific model.
    """
    # Filter results for this model
    model_results = df_results[df_results['model_name'] == model_name].copy()
    
    # Create mental illness set
    model_name_clean = model_name.replace(' ', '_')
    illness_set_uri = EX[f"DepressionSet_{model_name_clean}_{run_id}"]
    graph.add((illness_set_uri, RDF.type, MIAO.MentalIllnessesSet))
    graph.add((illness_set_uri, RDFS.label, 
               Literal(f"Depression severity detection results from {model_name}", lang="en")))
    graph.add((illness_set_uri, PROV.wasGeneratedBy, run_uri))
    graph.add((run_uri, PROV.generated, illness_set_uri))
    
    # Define Depression subclass (only once)
    depression_class = EX.Depression
    if (depression_class, RDFS.subClassOf, MIAO.MentalIllness) not in graph:
        graph.add((depression_class, RDFS.subClassOf, MIAO.MentalIllness))
        graph.add((depression_class, RDFS.label, Literal("Depression", lang="en")))
    
    # Create individual predictions
    for idx, row in model_results.iterrows():
        sample_id = row['sample_id']
        predicted_label = row['predicted_label']
        confidence = row['confidence']
        true_label = row['true_label']
        
        # Create illness instance
        illness_uri = EX[f"Depression_{model_name_clean}_{run_id}_sample_{sample_id}"]
        graph.add((illness_uri, RDF.type, depression_class))
        graph.add((illness_uri, MIAO.belongsToMentalIllnessesSet, illness_set_uri))
        graph.add((illness_set_uri, MIAO.hasMentalIllness, illness_uri))
        
        # Add category reference
        category_uri = category_map[predicted_label]
        graph.add((illness_uri, MIAO.referredToMentalIllnessCategory, category_uri))
        
        # Add severity level
        severity_name = severity_map[predicted_label]
        graph.add((illness_uri, MIAO.hasMentalIllnessLevel, Literal(severity_name)))
        
        # Add confidence
        graph.add((illness_uri, MIAO.hasMentalIllnessDetectionConfidence, 
                   Literal(float(confidence), datatype=XSD.decimal)))
        
        # Add sample reference
        sample_ref = f"depression_text_sample_{sample_id}"
        graph.add((illness_uri, MIAO.refersToSample, Literal(sample_ref, datatype=XSD.string)))
        
        # Add label
        is_correct = predicted_label == true_label
        status = "correct" if is_correct else "incorrect"
        graph.add((illness_uri, RDFS.label, 
                   Literal(f"{severity_name} depression prediction ({status}) by {model_name} for sample {sample_id}", lang="en")))
        
        # Add description with text snippet
        if sample_id < len(df_dataset):
            text_snippet = df_dataset.iloc[sample_id]['text'][:100]
            true_severity = severity_map[true_label]
            graph.add((illness_uri, DCTERMS.description, 
                       Literal(f"Predicted: {severity_name}, True: {true_severity}. Text: '{text_snippet}...'", lang="en")))
    
    return illness_set_uri

# Create predictions for all models
illness_sets = {}

for model_name in model_metrics.keys():
    illness_set_uri = create_predictions(
        g, df_results, df_dataset, model_name, 
        runs[model_name], run_ids[model_name], category_map
    )
    illness_sets[model_name] = illness_set_uri
    
    model_results_count = len(df_results[df_results['model_name'] == model_name])
    print(f"Created {model_results_count} predictions for: {model_name}")

print(f"\nTotal prediction sets created: {len(illness_sets)}")

Created 10 predictions for: BERT_Depression_Classifier
Created 10 predictions for: Feature_Framework_Classifier

Total prediction sets created: 2


## 9. Export RDF Graph

In [11]:
# Print statistics
print("\n" + "="*50)
print("RDF GRAPH STATISTICS")
print("="*50)
print(f"Total triples: {len(g)}")
print(f"\nTriples by type:")

# Count by type
type_counts = {}
for s, p, o in g.triples((None, RDF.type, None)):
    obj_str = str(o).split('#')[-1].split('/')[-1]
    type_counts[obj_str] = type_counts.get(obj_str, 0) + 1

for type_name, count in sorted(type_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {type_name}: {count}")


RDF GRAPH STATISTICS
Total triples: 357

Triples by type:
  Depression: 20
  ModelEvaluation: 10
  HyperParameter: 8
  EvaluationMeasure: 5
  MentalIllnessCategory: 4
  Software: 2
  Implementation: 2
  Model: 2
  AutomaticMentalIllnessesDetection: 2
  Run: 2
  MentalIllnessesSet: 2
  MentalIllnessesSchema: 1
  Dataset: 1


In [12]:
# Export to Turtle format
output_file = "depression_severity_experiment.ttl"
g.serialize(destination=output_file, format="turtle")
print(f"\nRDF graph exported to: {output_file}")
print(f"File size: {len(g.serialize(format='turtle'))} bytes")


RDF graph exported to: depression_severity_experiment.ttl
File size: 23471 bytes


## 10. Sample SPARQL Queries

In [13]:
# Query 1: Compare model performance
query1 = """
PREFIX mls: <http://www.w3.org/ns/mls#>
PREFIX miao: <https://w3id.org/miao#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?runLabel ?accuracy ?macroF1
WHERE {
  ?run a miao:AutomaticMentalIllnessesDetection ;
       rdfs:label ?runLabel .
  
  ?run mls:hasOutput ?accEval , ?f1Eval .
  
  ?accEval mls:specifiedBy ?accMeasure ;
           mls:hasValue ?accuracy .
  ?accMeasure rdfs:label "Accuracy"@en .
  
  ?f1Eval mls:specifiedBy ?f1Measure ;
          mls:hasValue ?macroF1 .
  ?f1Measure rdfs:label "Macro F1 Score"@en .
}
ORDER BY DESC(?accuracy)
"""

print("Query 1: Model Performance Comparison")
print("="*70)
results = g.query(query1)
for row in results:
    print(f"Model: {row.runLabel}")
    print(f"  Accuracy: {float(row.accuracy):.3f}")
    print(f"  Macro F1: {float(row.macroF1):.3f}")
    print()

Query 1: Model Performance Comparison
Model: Depression severity detection run: BERT_Depression_Classifier
  Accuracy: 0.900
  Macro F1: 0.880

Model: Depression severity detection run: Feature_Framework_Classifier
  Accuracy: 0.850
  Macro F1: 0.830



In [14]:
# Query 2: Distribution of predictions by severity level
query2 = """
PREFIX miao: <https://w3id.org/miao#>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT ?severityLevel (COUNT(?illness) as ?count)
WHERE {
  ?illness miao:hasMentalIllnessLevel ?severityLevel .
}
GROUP BY ?severityLevel
ORDER BY ?severityLevel
"""

print("Query 2: Prediction Distribution by Severity Level")
print("="*70)
results = g.query(query2)
for row in results:
    print(f"{row.severityLevel}: {row.count} predictions")

Query 2: Prediction Distribution by Severity Level
Mild: <built-in method count of ResultRow object at 0x7c68705d18f0> predictions
Minimal: <built-in method count of ResultRow object at 0x7c68708347c0> predictions
Moderate: <built-in method count of ResultRow object at 0x7c68705d18f0> predictions
Severe: <built-in method count of ResultRow object at 0x7c68708347c0> predictions


In [15]:
# Query 3: High confidence Severe depression predictions
query3 = """
PREFIX miao: <https://w3id.org/miao#>
PREFIX ex: <https://w3id.org/miao/depression-experiment#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?illness ?confidence ?sample
WHERE {
  ?illness a ex:Depression ;
           miao:referredToMentalIllnessCategory ex:Severe ;
           miao:hasMentalIllnessDetectionConfidence ?confidence ;
           miao:refersToSample ?sample .
  
  FILTER(?confidence >= 0.85)
}
ORDER BY DESC(?confidence)
"""

print("\nQuery 3: High-Confidence Severe Depression Cases")
print("="*70)
results = g.query(query3)
for row in results:
    print(f"Sample: {row.sample}")
    print(f"  Confidence: {float(row.confidence):.3f}")
    print()


Query 3: High-Confidence Severe Depression Cases
Sample: depression_text_sample_3
  Confidence: 0.910

Sample: depression_text_sample_7
  Confidence: 0.880

Sample: depression_text_sample_3
  Confidence: 0.870



In [16]:
# Query 4: Compare predictions between models for same samples
query4 = """
PREFIX miao: <https://w3id.org/miao#>
PREFIX prov: <http://www.w3.org/ns/prov#>

SELECT ?sample 
       ?severity1 ?confidence1 
       ?severity2 ?confidence2
WHERE {
  # First model prediction
  ?illness1 miao:hasMentalIllnessLevel ?severity1 ;
            miao:hasMentalIllnessDetectionConfidence ?confidence1 ;
            miao:refersToSample ?sample ;
            miao:belongsToMentalIllnessesSet ?set1 .
  
  # Second model prediction for same sample
  ?illness2 miao:hasMentalIllnessLevel ?severity2 ;
            miao:hasMentalIllnessDetectionConfidence ?confidence2 ;
            miao:refersToSample ?sample ;
            miao:belongsToMentalIllnessesSet ?set2 .
  
  FILTER(?set1 != ?set2)
  FILTER(?illness1 != ?illness2)
  FILTER(?severity1 != ?severity2)  # Only show disagreements
}
ORDER BY ?sample
"""

print("\nQuery 4: Model Disagreements on Severity Level")
print("="*70)
results = g.query(query4)
disagreement_count = 0
for row in results:
    disagreement_count += 1
    print(f"Sample: {row.sample}")
    print(f"  Model 1: {row.severity1} (confidence: {float(row.confidence1):.3f})")
    print(f"  Model 2: {row.severity2} (confidence: {float(row.confidence2):.3f})")
    print()

print(f"Total disagreements: {disagreement_count}")


Query 4: Model Disagreements on Severity Level
Sample: depression_text_sample_7
  Model 1: Severe (confidence: 0.880)
  Model 2: Moderate (confidence: 0.750)

Sample: depression_text_sample_7
  Model 1: Moderate (confidence: 0.750)
  Model 2: Severe (confidence: 0.880)

Total disagreements: 2


## 11. Validation Report

In [17]:
# Generate validation report
print("\n" + "="*70)
print("MIAO MODELING VALIDATION REPORT")
print("="*70)

# Check required components
num_models = len(model_metrics)
num_samples = len(df_dataset)
num_predictions = len(df_results)

checks = [
    ("Mental Illness Schema", len(list(g.triples((None, RDF.type, MIAO.MentalIllnessesSchema)))), 1),
    ("Mental Illness Categories", len(list(g.triples((None, RDF.type, MIAO.MentalIllnessCategory)))), 4),
    ("Dataset", len(list(g.triples((None, RDF.type, MLS.Dataset)))), 1),
    ("ML Implementations", len(list(g.triples((None, RDF.type, MLS.Implementation)))), num_models),
    ("Trained Models", len(list(g.triples((None, RDF.type, MLS.Model)))), num_models),
    ("Detection Runs", len(list(g.triples((None, RDF.type, MIAO.AutomaticMentalIllnessesDetection)))), num_models),
    ("Mental Illness Sets", len(list(g.triples((None, RDF.type, MIAO.MentalIllnessesSet)))), num_models),
    ("Individual Predictions", len(list(g.triples((None, MIAO.hasMentalIllnessDetectionConfidence, None)))), num_predictions),
    ("Model Evaluations", len(list(g.triples((None, RDF.type, MLS.ModelEvaluation)))), num_models * 5),
]

all_passed = True
for component, actual, expected in checks:
    status = "PASS" if actual >= expected else "FAIL"
    if status == "FAIL":
        all_passed = False
    print(f"[{status}] {component}: {actual}/{expected}")

print("\n" + "="*70)
if all_passed:
    print("VALIDATION PASSED: All MIAO components correctly modeled")
else:
    print("VALIDATION FAILED: Some components are missing or incomplete")
print("="*70)

# Additional statistics
print("\nAdditional Statistics:")
print(f"  Number of models: {num_models}")
print(f"  Number of samples: {num_samples}")
print(f"  Total predictions: {num_predictions}")
print(f"  Average predictions per model: {num_predictions / num_models:.1f}")


MIAO MODELING VALIDATION REPORT
[PASS] Mental Illness Schema: 1/1
[PASS] Mental Illness Categories: 4/4
[PASS] Dataset: 1/1
[PASS] ML Implementations: 2/2
[PASS] Trained Models: 2/2
[PASS] Detection Runs: 2/2
[PASS] Mental Illness Sets: 2/2
[PASS] Individual Predictions: 20/20
[PASS] Model Evaluations: 10/10

VALIDATION PASSED: All MIAO components correctly modeled

Additional Statistics:
  Number of models: 2
  Number of samples: 10
  Total predictions: 20
  Average predictions per model: 10.0
